In [22]:
from apiclient.discovery import build
from apiclient.errors import HttpError
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from newspaper import Article
import json
from string import digits
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.stem.porter import *
from collections import Counter 
from nltk.corpus import wordnet # To get words in dictionary with their parts of speech
from nltk.stem import WordNetLemmatizer # lemmatizes word based on it's parts of speech
from nltk.corpus import stopwords 
from textblob import TextBlob
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import cmudict
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from langdetect import detect
from numpy import dot
from numpy.linalg import norm
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
sys.path.append("/Users/rohanramesh/Documents/GitHub/Insight_writers/lib/")
from text_processing import ProcessArticle as pa
import suggestions as s
import networkx as nx

# Youtube video exploration
Write test code to perform named entity recognition on the title of articles and test using the Youtube api to search for videos to embed on webpage. Write the functions that will be within my web app for searching and parsing youtube content

In [ ]:
# Establish connection with mongo database
client = MongoClient()
mydb = client["testinsightdb"]

In [ ]:
# grab config file to get the api keys for youtube
# keys[0] apple, keys[1] listen notes
path_teams = '/Users/rohanramesh/Documents/Insight/data_bball_writers/config.txt'
with open(path_teams, 'r') as f:
    tmp = f.readlines()
keys = [i.rstrip() for i in tmp]

In [ ]:
# set up connection to communicate with youtube to search for videos
DEVELOPER_KEY = keys[0]
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

In [ ]:
# Need to be able to match team names with place and this fails with most entity recognition so 
# incorporating text file with all team names
# initialize spacy and load team names for named entity recognition
nlp = en_core_web_sm.load()
# team names txt file
path_teams = '/Users/rohanramesh/Documents/GitHub/Insight_writers/DashApp/assets/nba-teams.txt'
with open(path_teams, 'r') as f:
    tmp = f.readlines()
team_names = [i.rstrip() for i in tmp]

In [ ]:
def get_video_id(q, max_results,token, order="relevance", 
                 location=None, location_radius=None):
    """
    enter in search terms and get out metadate for youtube vido as well as the links to embed videos
    :param q: search terms
    :param max_results: max number of results back you want
    :param token: page search number set to none if want limited returns
    :param order: how to sort output (temporal vs relevance)
    :return: ydict: dictionary with metadata and embedding url
    :return: tok: token for next search
    """
    search_response = youtube.search().list(
        q=q, type="video", pageToken=token,part="id,snippet", 
        maxResults=max_results, location=location, 
        locationRadius=location_radius, safeSearch = 'strict').execute()
    videoId = []
    title = []
    description = []
    statistics = []
    embed_url = []
    tok = search_response['nextPageToken']

    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":
            title.append(search_result['snippet']['title'])
            videoId.append(search_result['id']['videoId'])
            response = youtube.videos().list(part='statistics, snippet, player', 
                                 id=search_result['id']['videoId']
                                ).execute()
            description.append(response['items'][0]['snippet']['description'])
            statistics.append(response['items'][0]['statistics'])
            embed_url.append(response['items'][0]['player']['embedHtml'])
            
    ydict = {'title':title,'videoId':videoId,
              'description':description,'stats':statistics,
             'embed_url':embed_url}
    return ydict, tok

def get_search_terms(string=None, team_names=None):
    """
    Do named entity recognition of title of article to enter into youtube search and
    incorporate in list of team names
    :param string: Title of article
    :param team_names: All team names
    :return: input_search: the string to enter to youtube for searching
    """
    doc = nlp(string)
    a = [i.text for i in doc.ents if (i.label_ == 'PERSON') | (i.label_ == 'GPE')]
    # iterate through each word in string and compare to team_names if any hits then keep
    if team_names is not None:
        tokens = word_tokenize(string)
        for i in tokens:
            if not difflib.get_close_matches(i,team_names, n=1):
                continue
            else:
                m = difflib.get_close_matches(i,team_names, n=1)[0]
            a.append(m)
    # now turn to normal string
    input_search = ['']
    for i in a:
        input_search[0] = input_search[0] + i + ' '
    return input_search[0]

def parse_iframe_html(input_vec):
    """
    parse the embedded url to pass to my web app
    incorporate in list of team names
    :param input_vec: url to be embedded from youtube
    :return: output_url: the exact string to embed
    """
    output_url = []
    for i in input_vec:
        idx = i.index('src="')
        tmpR = i[idx+5:]
        idx2 = tmpR.index('"')
        output_url.append(tmpR[:idx2])
    return output_url
        